<a href="https://colab.research.google.com/github/sacvik/sacvik/blob/main/GetDataCitationFromScientificPaper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [62]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

make_data_count_finding_data_references_path = kagglehub.competition_download('make-data-count-finding-data-references')

print(f'Data source import completed. find it here {make_data_count_finding_data_references_path}')


Data source import completed. find it here /root/.cache/kagglehub/competitions/make-data-count-finding-data-references


# 1. Introduction

In [56]:
!pip install fitz tools etree

ERROR: Could not find a version that satisfies the requirement etree (from versions: none)
ERROR: No matching distribution found for etree


In [16]:
### import required modules
import numpy as np
import pandas as pd
import fitz  # PyMuPDF
import os
from pathlib import Path
from lxml import etree
import re

In [36]:
## setting up paths
train_data_file_path =  make_data_count_finding_data_references_path + '/train/'
output_dir_path = 'output/'

In [35]:
os.listdir(train_data_file_path)

['PDF', 'XML']

# 2. Extraction of data files

In [22]:
def parse_pdf_or_xml(pdf_path, xml_dir, output_dir):
    """
    Check for a PDF file and its corresponding XML file. Parse XML if available, otherwise parse PDF using PyMuPDF.

    Args:
        pdf_path (str): Path to the PDF file.
        xml_dir (str, optional): Directory containing XML files. Defaults to same directory as PDF.
        output_dir (str, optional): Directory to save extracted text. If None, no file is saved.

    Returns:
        Dictionary with section names (e.g., Methods, References) and their text content.
    """
    try:
        # Validate PDF path
        pdf_path = Path(pdf_path)
        if not pdf_path.exists() or pdf_path.suffix.lower() != '.pdf':
            return {}

        # Determine XML directory (default to PDF's directory if not specified)
        xml_dir = Path(xml_dir) if xml_dir else pdf_path.parent
        if not xml_dir.exists():
            xml_dir = None

        # Check for corresponding XML file (same name as PDF but with .xml extension)
        xml_path = xml_dir / pdf_path.with_suffix('.xml').name if xml_dir else None
        if xml_path and xml_path.exists():
            extracted_data = parse_xml_file(str(xml_path))
        else:
            extracted_data = parse_pdf_file(str(pdf_path))

        # Save output if output_dir is specified
        if output_dir and extracted_data:
            Path(output_dir).mkdir(parents=True, exist_ok=True)
            output_path = Path(output_dir) / f"{pdf_path.stem}_extracted.txt"
            save_extracted_text(extracted_data, str(output_path))

        return extracted_data

    except Exception:
        return {}

def parse_xml_file(xml_path):
    """
    Parse XML file to extract text from relevant sections (e.g., Methods, References).

    Args:
        xml_path (str): Path to the XML file.

    Returns:
        Dictionary with section names and their text content.
    """
    try:
        # Parse XML
        tree = etree.parse(xml_path)
        root = tree.getroot()

        # Namespace for JATS (adjust if using TEI or other schema)
        ns = {'jats': 'http://jats.nlm.nih.gov'}

        output = {}

        # Define sections to extract (extend as needed)
        sections = {
            'Abstract': '//jats:abstract',
            'Methods': '//jats:sec[@sec-type="methods" or @sec-type="materials|methods"]',
            'References': '//jats:ref-list'
        }

        for section_name, xpath in sections.items():
            elements = root.xpath(xpath, namespaces=ns)
            section_text = []
            for elem in elements:
                text = ''.join(elem.itertext()).strip()
                if text:
                    section_text.append(text)
            if section_text:
                output[section_name] = section_text

        return output

    except Exception:
        return {}

def parse_pdf_file(pdf_path):
    """
    Extract text from a PDF file using PyMuPDF, organizing by potential sections.

    Args:
        pdf_path (str): Path to the PDF file.

    Returns:
        Dictionary with section names and their text content.
    """
    try:
        # Open PDF
        doc = fitz.open(pdf_path)
        output = {}

        # Common section headers for scientific papers
        section_patterns = [
            r'^(Abstract|Introduction|Methods|Materials and Methods|Results|Discussion|References|Bibliography)\s*$',
            r'^\d+\.\s*(Abstract|Introduction|Methods|Materials and Methods|Results|Discussion|References|Bibliography)\s*$'
        ]

        current_section = "Unknown"

        for page_num in range(len(doc)):
            page = doc[page_num]
            text = page.get_text("text")
            if not text.strip():
                continue

            # Split text into lines for section detection
            lines = text.split('\n')
            page_text = []

            for line in lines:
                line = line.strip()
                if not line:
                    continue

                # Check for section headers
                is_section = False
                for pattern in section_patterns:
                    if re.match(pattern, line, re.IGNORECASE):
                        current_section = re.sub(r'^\d+\.\s*', '', line).strip()
                        is_section = True
                        break

                if not is_section:
                    page_text.append(line)

            # Store text under current section
            if current_section not in output:
                output[current_section] = []
            output[current_section].extend(page_text)

        doc.close()
        return output

    except Exception:
        return {}

def save_extracted_text(extracted_data, output_path):
    """
    Save extracted text to a file.

    Args:
        extracted_data: Extracted text organized by sections.
        output_path (str): Path to save the output file.
    """
    try:
        if not os.path.exists(output_path):
            os.mkdir(output_path)
        with open(output_path, 'w', encoding='utf-8') as f:
            for section, lines in extracted_data.items():
                f.write(f"## {section}\n")
                f.write('\n'.join(lines))
                f.write('\n\n')
    except Exception:
        pass

def process_directory(input_dir, xml_dir, output_dir):
    """
    Process all PDFs in a directory, checking for corresponding XML files.

    Args:
        input_dir (str): Directory containing PDF files.
        xml_dir (str, optional): Directory containing XML files. Defaults to input_dir.
        output_dir (str, optional): Directory to save extracted text files.

    Returns:
        List of extracted data dictionaries for all processed files.
    """
    results = []
    input_dir = Path(input_dir)

    for pdf_file in input_dir.glob("*.pdf"):
        extracted_data = parse_pdf_or_xml(pdf_file, xml_dir, output_dir)
        if extracted_data:
            results.append(extracted_data)

    return results

In [37]:
input_directory = train_data_file_path + "PDF"
xml_directory = train_data_file_path + "XML"
output_directory = output_dir_path + "extracted_data/"

display(input_directory)
display(xml_directory)
display(output_directory)

'/root/.cache/kagglehub/competitions/make-data-count-finding-data-references/train/PDF'

'/root/.cache/kagglehub/competitions/make-data-count-finding-data-references/train/XML'

'output/extracted_data/'

In [38]:
input_dir = input_directory
xml_dir = xml_directory
output_dir = output_directory

In [40]:
results = []
input_dir = Path(input_dir)

In [42]:
for pdf_file in input_dir.glob("*.pdf"):
    pass

In [44]:
pdf_path = pdf_file
pdf_path

PosixPath('/root/.cache/kagglehub/competitions/make-data-count-finding-data-references/train/PDF/10.1186_s12863-017-0513-7.pdf')

In [46]:
pdf_path = Path(pdf_path)

In [47]:
xml_dir = Path(xml_dir) if xml_dir else pdf_path.parent
if not xml_dir.exists():
    xml_dir = None
xml_path = xml_dir / pdf_path.with_suffix('.xml').name if xml_dir else None
if xml_path and xml_path.exists():
    extracted_data = parse_xml_file(str(xml_path))
else:
    extracted_data = parse_pdf_file(str(pdf_path))

In [59]:
from lxml import etree

def parse_xml(file_path):
    # Parse the XML file
    tree = etree.parse(file_path)
    root = tree.getroot()

    # Recursive function to print tags and text
    def print_elements(element, indent=0):
        if element.text and element.text.strip():
            print("  " * indent + f"{element.tag}: {element.text.strip()}")
        for child in element:




In [60]:
# Path to your uploaded XML file
aa = parse_xml(xml_path)


  <cyfunction ProcessingInstruction at 0x7b56cc9bf6b0>: open_access
  <cyfunction ProcessingInstruction at 0x7b56cc9bf6b0>: -//Springer-Verlag//DTD A++ V2.4//EN
  <cyfunction ProcessingInstruction at 0x7b56cc9bf6b0>: public
  <cyfunction ProcessingInstruction at 0x7b56cc9bf6b0>: A++V2.4.dtd
  <cyfunction ProcessingInstruction at 0x7b56cc9bf6b0>: 2.4
  <cyfunction ProcessingInstruction at 0x7b56cc9bf6b0>: springer2nlmx2.xsl
  <cyfunction ProcessingInstruction at 0x7b56cc9bf6b0>: 1
      journal-id: BMC Genet
      journal-id: BMC Genet
        journal-title: BMC Genetics
      issn: 1471-2156
        publisher-name: BioMed Central
        publisher-loc: London
      article-id: 5429570
      article-id: 513
      article-id: 10.1186/s12863-017-0513-7
          subject: Research Article
        article-title: High-content behavioral profiling reveals neuronal genetic network modulating
          italic: Drosophila
            surname: Aleman-Meza
            given-names: Boanerges
      

In [54]:
xml_path

PosixPath('/root/.cache/kagglehub/competitions/make-data-count-finding-data-references/train/XML/10.1186_s12863-017-0513-7.xml')

In [50]:
tree = etree.parse(xml_path)
root = tree.getroot()

In [ ]:
a

In [51]:
# Namespace for JATS (adjust if using TEI or other schema)
ns = {'jats': 'http://jats.nlm.nih.gov'}

output = {}

# Define sections to extract (extend as needed)
sections = {
    'Abstract': '//jats:abstract',
    'Methods': '//jats:sec[@sec-type="methods" or @sec-type="materials|methods"]',
    'References': '//jats:ref-list'
}


In [ ]:
# def main():
#     # Example usage
#     input_directory = "path/to/pdf/folder"  # Replace with your PDF directory
#     xml_directory = "path/to/xml/folder"    # Replace with your XML directory (or None)
#     output_directory = "path/to/output/folder"  # Replace with your output directory

#     # Process all PDFs in the directory
#     results = process_directory(input_directory, xml_directory, output_directory)

#     # Optional: Print results for verification
#     for result in results:
#         print(result)

# if __name__ == "__main__":
#     main()

In [ ]:
output_path = output_dir_path + 'extracted_data/'

In [ ]:
import os
os.mkdir(output_path)

'/kaggle/working/output/extracted_data/'

In [ ]:
pd1 = pd.read_csv(train_data_file_path + 'train_labels.csv')
pd1.columns

In [ ]:
os.getenv()